Normalização do modelo BCS não linear

In [1]:
import matplotlib.pyplot as plt
import numpy as np
exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
exec(compile(open('subrotinas.py', "rb").read(), 'subrotinas.py', 'exec'))
%matplotlib tk

Dados carregados
Dados carregados


### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>



Definindo os fatores de ajuste de escala a partir dos limites operacionais

In [2]:
# Valores máximos e mínimos para normalização
#Entradas - conforme binder e pavlov
def Lim_c(x):
    return x[1]-x[0]
f_lim=(30,75)
zclim=(0,1)
pmlim=(1e5,50e5)
pbhlim=(1e5,pr) 
pwhlim=(1e5,50e5) 
qlim=(35/3600,65/3600)
pbc=Lim_c(pbhlim)
pwc=Lim_c(pwhlim)
qc=Lim_c(qlim)
pbmin=pbhlim[0]
pwmin=pwhlim[0]
qmin=qlim[0]
H_lim=(-136.31543417849096, 1420.7697113483912)
qch_lim=(0.0, 0.03290348005910621)



In [3]:
pm=20e5; #Simplificando pm fixo
#pbh  - pressão na bomba
#pwh - Pressão de fundo do poço,
#q - vazão
#PI índice de produtividade do poço
#PinC  pressão na choke
# Pressão de manifold

Definindo as variáveis simbólicas

In [4]:
# Criando simbolica
from casadi import *
nx = 5; nu = 2;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena
pbh = x[0]
pwh = x[1]
q = x[2]
fq = x[3]; zc = x[4];

In [5]:
# Entradas

fqref = u[0]; zcref = u[1];
pm=2e6;
# zc=zcref 
# fq=fqref

In [6]:
# Calculo do HEAD e delta de press�o
q0 = (q*qc+qmin) / Cq * (f0 / fq)
H0 = -1.2454e6 * q0 ** 2 + 7.4959e3 * q0 + 9.5970e2
H = CH * H0 * (fq / f0) ** 2  # Head
#Pp = rho * g * H  # Delta de press�o

# Calculo da Potencia e corrente da bomba
P0 = -2.3599e9 * q0 ** 3 - 1.8082e7 * q0 ** 2 + 4.3346e6 * q0 + 9.4355e4
P = Cp * P0 * (fq / f0) ** 3;  # Potencia
I = Inp * P / Pnp  # Corrente

# Calculo da press�o de intake
F1 = 0.158 * ((rho * L1 * ((q*qc+qmin)) ** 2) / (D1 * A1 ** 2)) * (mu / (rho * D1 * ((q*qc+qmin)))) ** (1 / 4)
F2 = 0.158 * ((rho * L2 * ((q*qc+qmin)) ** 2) / (D2 * A2 ** 2)) * (mu / (rho * D2 * ((q*qc+qmin)))) ** (1 / 4)
pin = pbh*pbc+pbmin - rho * g * h1 - F1
# Vazao do reservatorio e vazao na choke
qr = PI * (pr - (pbh*pbc+pbmin))
qch = (zc/100)*Cc * sqrt(fabs(pwh*pwc+pwmin - pm));

# Termos não lineares
#menor q implica em menor F

funcF1=Function('funcF1',[x],[F1])
funcF2=Function('funcF2',[x],[F2])
F1lim=(funcF1([0,0,qlim[0],0,0]),funcF1([0,0,qlim[1],0,0]))
F2lim=(funcF2([0,0,qlim[0],0,0]),funcF2([0,0,qlim[1],0,0]))
F1c=Lim_c(F1lim)
F2c=Lim_c(F2lim)
Hc=Lim_c(H_lim)
qcc=Lim_c(qch_lim)
print('Limites: pbh,pwh,q')
print(pbhlim,pwhlim,qlim)
print('Limites: F1,F2,H,qch')
print(F1lim,F2lim,H_lim,qch_lim)


Limites: pbh,pwh,q
(100000.0, 12600000.0) (100000.0, 5000000.0) (0.009722222222222222, 0.018055555555555554)
Limites: F1,F2,H,qch
(DM(435476), DM(440889)) (DM(1.04514e+06), DM(1.05813e+06)) (-136.31543417849096, 1420.7697113483912) (0.0, 0.03290348005910621)


In [7]:
# F1c=941799.5331
# F2c=2260318.8795
#qcc=0.033987702
#Hc=1511.97
#Normalizar termos não lineares
##########################
qch=(qch-qch_lim[0])/qcc
F1=(F1-F1lim[0])/F1c
F2=(F2-F2lim[0])/F2c
H=(H-H_lim[0])/Hc
###########################

#pr*b1*PI/V1
# qch=qch*qcc
# F1=F1c*F1
# F2=F2c*F2
# H=Hc*H
dpbhdt = (1/pbc)*b1/V1*(qr - (q*qc+qmin))
dpwhdt = (1/pwc)*b2/V2*((q*qc+qmin) - (qcc*qch+qch_lim[0]))
dqdt = (1/(qc*M))*(pbh*pbc+pbmin - (pwh*pwc+pwmin) - rho*g*hw - (F1c*F1+F1lim[0]) - (F2c*F2+F2lim[0]) + rho * g * (H*Hc+H_lim[0]))
dfqdt = (fqref - fq)/tp[0];
dzcdt = (zcref - zc)/tp[1];

dxdt = vertcat(dpbhdt,dpwhdt,dqdt,dfqdt,dzcdt);

# Restricao do Elemento Final
dudt = vertcat(if_else(fabs(dfqdt)>dudt_max[0],sign(dfqdt)*dudt_max[0],dfqdt),
       if_else(fabs(dzcdt)>dudt_max[1],sign(dzcdt)*dudt_max[1],dzcdt));

In [8]:
xss = np.float32(np.array([8311024.82175957,2990109.06207437,0.00995042241351780,50,50]))
x0=np.array([pbmin,pwmin,qmin,0,0])
xc=np.array([pbc,pwc,qc,1,1])
xssn = (xss-x0)/xc


In [9]:
uss = np.array([50,50])


In [10]:
# Função casadi
#dxdt = casadi.vertcat(dpbhdt,dpwhdt,dqdt) 
Eq_Estado = casadi.Function('Eq_Estado',[x,u,dudt_max],[vertcat(dpbhdt,dpwhdt,dqdt,dudt)],
                     ['x','u','dudt_max'],['dxdt'])

y=vertcat(pin,H);
ny = y.size1()
# Equações algébricas
sea_nl = casadi.Function('sea_nl',[x,u],[y,pin,H],\
                 ['x','u'],['y','pin','H']); # Sistema de Eq. Algebricas variaveis de sa�da

BCS={
     'x': x,
     'u': u,
     'y': y,
     'nx': nx,
     'nu': nu,
     'ny': ny,
     'NaoLinear': {'sedo_nl': Eq_Estado(x,u,[dfq_max,dzc_max]),
                   'sea_nl': sea_nl
                   }
}
#%% Calculo do estacionario
#% Func��o objetivo
dxdt_0 = Eq_Estado(BCS['x'], BCS['u'],[dfq_max,dzc_max]);
J = sum1(dxdt_0**2);
#% Otimizador
opt={
     'ipopt':{
         'print_level':0,
         'acceptable_tol':1e-8,
         'acceptable_obj_change_tol':1e-6,
         'max_iter':50
         },
     'print_time':0,
     }

opt['ipopt']['print_level']=0;# %0,3
opt['print_time']=0;
opt['ipopt']['acceptable_tol']=1e-8;
opt['ipopt']['acceptable_obj_change_tol']=1e-6;
opt['ipopt']['max_iter']=50;



In [11]:
MMQ = {'x':BCS['x'], 'f':J, 'p':BCS['u']}
#nlp={'x':vertcat(BCS['x'],BCS['u']), 'f':J} #variáveis de decisão, função f, g (N/A)
#nlp={'x':BCS['x'], 'f':J}
solver = nlpsol('solver', 'ipopt', MMQ, opt)
# Restrições das variaveis de decis�o
# minimo
args={
      'lbx': np.zeros((nx,1)),
# m�ximo
      'ubx':np.full((nx, 1), np.inf)
      }

# Solu��o do otimizador
sol=solver(x0=BCS['x'], lbx=args['lbx'], ubx=args['ubx'], p=BCS['u']);
yss=sea_nl(sol['x'],BCS['u'])
Estacionario = Function('Estacionario',[BCS['x'],BCS['u']],\
    [sol['x'],yss[0]],\
    ['x0','uss'],['xss','yss']);

BCS['Estacionario'] = Estacionario;
f_ss,zc_ss= (np.array([50, 50]))
uss = np.array([f_ss,zc_ss]); # Entradas do estacionario
# uss_n=normalizar_u(uss,unorm)
# uss_n

sol=solver(x0=xssn, p=uss);

sol['x']*xc+x0
#sol['x']*xc




******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



DM([8.31102e+06, 2.99011e+06, 0.00995042, 50, 50])

In [12]:
# Definir variaveis manipuladas e controladas e disturbio externo
mv = [0,1]    #% [f, Zc]
pv = [0,1]  #% [pin, H]  #% [P, I]
#pv = [2,3];  #% [pin, H]
de = 2;      #% [pm]
tg = 2;      #% MV target
#% Parametros
ts = 1;
#%Modelo de predição
#% Criando o objeto para predição do modelo
# Iniciando variavel dicionário para a construção da EDO
# sedo = {'x': BCS['x'][0:3], # Estados
#         'p': BCS['u'], #Variáveis exogenas
#         'ode': BCS['NaoLinear']['sedo_nl'] # SEDO (Gerado no bcs_settings)
#         };

sedo = {'x': BCS['x'], # Estados
        'p': BCS['u'], #Variáveis exogenas
        'ode': Eq_Estado(x,u,[dfq_max,dzc_max]) # SEDO (Gerado no bcs_settings)
        };

#% Criando o objeto p,ra integração da Eq_estado
opt = {'tf':ts,
       't0':0

       };   #% opções do integrador

In [13]:
int_odes = integrator('int_odes','cvodes',sedo,opt);
# objeto integrador
res = int_odes(x0=BCS['x'],p=BCS['u']);             #   % solução um passo a frente
npv = len(pv); nmv = len(mv);

In [14]:
# Criando o objeto para solução da equação de medição
Eq_medicao = Function('Eq_medicao',[BCS['x'],BCS['u']],[BCS['y'][pv]],['x','u'],['y']);
# Criacao do objeto para simulacao do BCS Eq de estado + Eq de Medicao
Modelo_Predicao = Function('Modelo_Predicao',[BCS['x'],BCS['u']],[res['xf'],Eq_medicao(res['xf'],BCS['u'])],['xk_1','uk_1'],['xk','yk']);
Modelo_Predicao2 = Function('Modelo_Predicao2',[BCS['x'],BCS['u']],[res['xf']],['xk_1','uk_1'],['xk']);

In [15]:
xpk = xss;
uss
#xpk=normalizar(xss,xnorm)

#uk_1 = normaliza_u(uss[mv],unorm)
#uk_1=(uss[mv]-unorm[:,0])/unorm[:,1]
# MVS
# Aloca��o de variaveis
Xk = np.zeros((nx,1))

In [16]:
Yk = np.zeros((npv,1))
Uk = np.zeros((nmv,1))
Ymk = Yk
Ys = Yk
Ymin = Yk
Ymax = Yk

In [17]:
tsim = 200;
nsim=int(round(tsim/ts)+1)
xss_n=xssn

In [18]:
#Valores iniciais de simulação
tss=5 # Tempo inicial no estacionário
Uk= uss
# Povoar vetor de X com dados não normalizados


In [19]:
xpk = Modelo_Predicao2(xssn,uss) #valores normalizados
ypk=sea_nl(xpk,uss)[0]
Yk=ypk*np.array([1,Hc])+[0,H_lim[0]]
ypk[1]*Hc+H_lim[0]

DM(592.126)

In [20]:
xpk*xc+x0

DM([8.31102e+06, 2.99011e+06, 0.00995042, 50, 50])

In [21]:
uk_1 = np.array([[60], [50]]);
nstep = tsim
a_range = [35,65]
b_range = [30,50]

u_f,prbs=APRBS(a_range,b_range,nstep)

dados = np.load('BCS_data_train_limitado.npz')
u_f=dados['fk']
uk_1 = np.array([u_f,np.ones_like(u_f)*50]);
xi=(np.arange(0,int(nsim*ts)-1,ts));
#xi=np.arange(0,10,ts)
xi.shape
Xk=xpk*xc+x0
uss
Modelo_Predicao2(xssn,uss)

DM([0.656882, 0.589818, 0.027384, 50, 50])

In [22]:
print("Simulação iniciada")

#for k in range(1,5):
for k in range(1,nsim-1):
    xpk = Modelo_Predicao2(xpk,uk_1[:,k])
    Xk = hcat([Xk,xpk*xc+x0]) #desnormalizar x e preencher vetor
    ypk=sea_nl(xpk,uk_1[:,k])[0]
    Yk = hcat([Yk,ypk*np.array([1,Hc])+[0,H_lim[0]]]);
    Uk = hcat([Uk,uk_1[:,k]])

print("Xk shape ="+str(Xk.shape))
# print("Uk shape ="+str(Uk.shape))
# print("Yk shape ="+str(Uk.shape))
# print("xi shape ="+str(xi.shape))
print("Ok.simulação concluida")
Xk[0,:].shape

Simulação iniciada
Xk shape =(5, 200)
Ok.simulação concluida


(1, 200)

In [23]:
fig1=plt.figure()
label = [r'$p_{in}(bar)$','H(m)','P','I','qc','qr' ];
for iy in range(0,npv):
    ax = fig1.add_subplot(npv,1,iy+1)
    #print(iy)
    if iy == 0: # Pin
        ax.plot(xi,(Yk[iy,:].T)/1e5, label='Medição')
        #ax.plot(xi,Ymk[iy,:].T/1e5, label='EKF')
        ax.set_ylabel(label[iy])
        ax.set(xlim=(xi[0], nsim*ts))
       # ax.set(ylim=(40,62))
        plt.grid(True)
    else: # H
        ax.plot(xi,Yk[iy,:].T, label='Medição')
        #ax.plot(xi,Ymk[iy,:].T,label='EKF')
        ax.set_ylabel(label[iy])
        ax.set(xlim=(xi[0], nsim*ts))
       # ax.set(ylim=(580, 850))
        plt.grid(True)
#ax.plot(xi,Yk[2,:].T, label='EKF')
ax.legend();
ax.set_xlabel('Time (nT)')
fig1.show()


In [24]:
# #% Restrição
umin  = np.array([35, 0]); np.transpose(umin);  # lower bounds of inputs
umax  = np.array([65, 100]); np.transpose(umax); # upper bounds of inputs
dumax = np.array([0.5, dzc_max]); np.transpose(dumax);
fig2=plt.figure()
label = ['f(Hz)',r'$z_c$(%)'];
for iu in range(0,nmv):
    ax2=fig2.add_subplot(nmv,1,iu+1)
    ax2.plot(xi,Uk[iu,:].T, label='Medição')
    ax2.plot([1,nsim],[umin[iu], umin[iu]],'--r')
    ax2.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
    ax2.set_ylabel(label[iu])
    ax2.set(xlim=(xi[0], nsim*ts))
    if iu==0:
        ax2.set(ylim=(30, 70))

    plt.grid(True)


In [25]:
fig3=plt.figure()
label = ['Pbh (bar)','Pwh (bar)','q(m3/s)'];
for iu in range(0,3):
    ax3=fig3.add_subplot(3,1,iu+1)
    if iu==2:
        ax3.plot(xi,(Xk[iu,:].T/xc[iu]), label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)
    else:
        ax3.plot(xi,Xk[iu,:].T/xc[iu], label='Medição')
        #ax3.plot([1,nsim],[umin[iu], umin[iu]],'--r')
        #ax3.plot([1,nsim],[umax[iu], umax[iu]],'--r', label='Restrição')
        ax3.set_ylabel(label[iu])
        #ax3.set(xlim=(xi[0], nsim*ts))
        # if iu==0:
        #     #ax2.set(ylim=(30, 70))
        #     print(iu)
        plt.grid(True)


In [26]:
# Xk[2,:]=Xk[2,:]*xc[2] #desnormalizar vazão
# Yk[1,:]=Yk[1,:]*Hc #desnormalizar Head

In [27]:
exec(compile(open('envelope.py', "rb").read(), 'envelope.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
fig4,ax4=plt.subplots()
plt.grid(True)
BCS['Envelope']['fig'](ax4); # grafico do envelope
#
# Evolução dentro do envelope
ax4.plot(Xk[2,0:].T*3600,Yk[1,0:].T,'--k')
ax4.plot(Xk[2,0]*3600,Yk[1,0],'o')#,'MarkerFaceColor',[0,1,0],'MarkerEdgeColor',[0,0,0])
ax4.plot(Xk[2,-1]*3600,Yk[1,-1],'o')#,'MarkerFaceColor',[1,0,0],'MarkerEdgeColor',[0,0,0])
ax4.annotate('t=0',
             xy=(float(Xk[2,0]*3600),float(Yk[1,0])),
             xytext=(float(Xk[2,0]*3600)-5,float(Yk[1,0])+10),
             arrowprops=dict(facecolor='green', shrink=0.01))

ax4.annotate('t='+str(nsim),
             xy=(float(Xk[2,-1]*3600),float(Yk[1,-1])),
             xytext=(float(Xk[2,-1]*3600)-7,float(Yk[1,-1])+10),
             arrowprops=dict(facecolor='red', shrink=0.01))
plt.show()

#np.savez('BCS_data_train_limitado.npz', t=xi, x1=Xk[0,:].T,x2=Xk[1,:].T,x3=Xk[2,:].T,x4=Xk[3,:].T,x5=Xk[4,:].T)

Envelope carregado
